# ECC-based Lightweight Authentication Scheme for Smart Grid

This notebook implements a lightweight authentication and key agreement protocol based on elliptic curve cryptography (ECC), suitable for smart grid communications.  
It simulates the **initialization, registration, and mutual authentication phases** and demonstrates how both parties establish a shared session key securely.

#### Referenced Research Paper:  
[Design and hardware implementation of a security-enhanced elliptic curve cryptography based lightweight authentication scheme for smart grid communications](https://doi.org/10.1016/j.future.2018.02.034)  


In [25]:
import hashlib
import secrets
from tinyec import registry

# Hash Functions
def H1(data: bytes, q: int) -> int:
    return int.from_bytes(hashlib.sha256(data).digest(), 'big') % q

def H2(data: bytes, q: int) -> int:
    return int.from_bytes(hashlib.sha3_256(data).digest(), 'big') % q

def H3(data: bytes, q: int) -> int:
    return int.from_bytes(hashlib.blake2s(data).digest(), 'big') % q


# TrustedAnchor Class

class TrustedAnchor:
    def __init__(self):
        # Curve selection
        self.curve = registry.get_curve('secp192r1')
        self.p = self.curve.field.p
        self.a = self.curve.a
        self.b = self.curve.b
        self.P = self.curve.g
        self.q = self.curve.field.n

        # TA private/public key
        self.sk_TA = secrets.randbelow(self.q - 1) + 1
        self.PK_TA = self.sk_TA * self.P

        # public parameters
        self.public_params = {
            "curve_eq": f"y² = x³ + {self.a}x + {self.b} mod {self.p}",
            "p": self.p,
            "q": self.q,
            "P": (self.P.x, self.P.y),
            "PK_TA": (self.PK_TA.x, self.PK_TA.y),
            "H1": "SHA-256",
            "H2": "SHA3-256",
            "H3": "BLAKE2s"
        }

    def get_public_params(self):
        return self.public_params

    def process_registration(self, idx, Rx_point):

        # Step 2.a: Generate rTA
        rTA = secrets.randbelow(self.q - 1) + 1

        # Step 2.b: Compute RTA = rTA * P
        RTA = rTA * self.P

        # Step 2.b: Compute WTx = Rx + RTA
        WTx = Rx_point + RTA

        # Step 2.c: Compute ex = H1(idx || WTx)
        idx_bytes = idx.encode()
        WTx_bytes = WTx.x.to_bytes(24, 'big') + WTx.y.to_bytes(24, 'big')
        ex = H1(idx_bytes + WTx_bytes, self.q)

        # Step 2.d: Compute yx = rTA + ex * sk_TA mod q
        yx = (rTA + ex * self.sk_TA) % self.q

        # TA sends {yx, WTx} back to Ex
        return yx, WTx

class Entity:
    def __init__(self, idx, TA: TrustedAnchor):
        self.idx = idx
        self.TA = TA
        self.q = TA.q
        self.P = TA.P
        self.PK_TA = TA.PK_TA

    def register(self):
        # Step 1.a: Choose random rx ∈ Z*q
        self.rx = secrets.randbelow(self.q - 1) + 1

        # Step 1.b: Compute Rx = H1(rx || idx) · P
        rx_bytes = self.rx.to_bytes(24, 'big')
        idx_bytes = self.idx.encode()
        hval = H1(rx_bytes + idx_bytes, self.q)
        self.Rx = hval * self.P

        # Send {idx, Rx} to TA and receive {yx, WTx}
        self.yx, self.WTx = self.TA.process_registration(self.idx, self.Rx)

        # Step 3: Compute skx = H1(rx || idx) + yx mod q
        self.skx = (hval + self.yx) % self.q

        # Compute PKx = skx · P
        self.PKx = self.skx * self.P

        # Verify PKx = WTx + H1(idx || WTx) · PK_TA
        WTx_bytes = self.WTx.x.to_bytes(24, 'big') + self.WTx.y.to_bytes(24, 'big')
        ex = H1(self.idx.encode() + WTx_bytes, self.q)
        PKx_check = self.WTx + ex * self.PK_TA

        if (self.PKx.x == PKx_check.x) and (self.PKx.y == PKx_check.y):   # Theorem 1
            print(f"[SUCCESS] Entity {self.idx} public key verified.")
        else:
            print(f"[FAILURE] Entity {self.idx} public key verification failed.")

        return self.PKx


    # Authentication Phase as Initiator (EA)
 

    def initiate_auth(self, EB_entity):
        # Step 1: Choose rA
        self.rA = secrets.randbelow(self.q - 1) + 1
        RA = self.rA * self.P

        # Send idA, RA, WTA to EB
        return EB_entity.respond_auth(
            idA=self.idx,
            RA=RA,
            WTA=self.WTx,
            sender=self
        )

    
    # Authentication Phase as Responder (EB)

    def respond_auth(self, idA, RA, WTA, sender):
        # Step 2:
        self.rB = secrets.randbelow(self.q - 1) + 1
        RB = self.rB * self.P

        # Compute H1(idA || WTA)
        idA_bytes = idA.encode()
        WTA_bytes = WTA.x.to_bytes(24, 'big') + WTA.y.to_bytes(24, 'big')
        h_val = H1(idA_bytes + WTA_bytes, self.q)

        # Compute WTA + H1(idA || WTA) * PK_TA
        temp = WTA + h_val * self.PK_TA

        # Compute partial session key KA_B
        KA_B = self.rB * temp + self.skx * RA

        # Compute VB = H2(KA_B || idB || RB || WTB)
        KA_B_bytes = KA_B.x.to_bytes(24, 'big') + KA_B.y.to_bytes(24, 'big')
        RB_bytes = RB.x.to_bytes(24, 'big') + RB.y.to_bytes(24, 'big')
        idB_bytes = self.idx.encode()
        WTB_bytes = self.WTx.x.to_bytes(24, 'big') + self.WTx.y.to_bytes(24, 'big')

        VB = H2(KA_B_bytes + idB_bytes + RB_bytes + WTB_bytes, self.q)

        # Send idB, RB, VB, WTB to EA
        return sender.finalize_auth(
            idB=self.idx,
            RB=RB,
            VB=VB,
            WTB=self.WTx,
            RA=RA,
            rA=sender.rA
        )

   
    # Finalize Authentication as EA
   

    def finalize_auth(self, idB, RB, VB, WTB, RA, rA):
        # Compute H1(idB || WTB)
        idB_bytes = idB.encode()
        WTB_bytes = WTB.x.to_bytes(24, 'big') + WTB.y.to_bytes(24, 'big')
        h_val = H1(idB_bytes + WTB_bytes, self.q)

        # Compute WTB + H1(idB || WTB) · PK_TA
        temp = WTB + h_val * self.PK_TA

        # Compute partial session key KB_A
        KB_A = rA * temp + self.skx * RB

        # Compute verifier on EA side
        KB_A_bytes = KB_A.x.to_bytes(24, 'big') + KB_A.y.to_bytes(24, 'big')
        RB_bytes = RB.x.to_bytes(24, 'big') + RB.y.to_bytes(24, 'big')
        idB_bytes = idB.encode()
        WTB_bytes = WTB.x.to_bytes(24, 'big') + WTB.y.to_bytes(24, 'big')

        VB_check = H2(KB_A_bytes + idB_bytes + RB_bytes + WTB_bytes, self.q)

        if VB_check != VB:
            print("[ERROR] VB verification failed at EA. Session terminated.")
            return None

        # Compute shared session key
        idA_bytes = self.idx.encode()
        SSKAB = H3(idA_bytes + idB_bytes + KB_A_bytes, self.q)

        # Compute VA = H2(KB_A || idA)
        VA = H2(KB_A_bytes + idA_bytes, self.q)

        # Send VA back to EB for final verification
        return {
            "EA_entity": self,
            "SSKAB": SSKAB,
            "idB": idB,
            "RB": RB,
            "VA": VA,
            "KB_A": KB_A
        }


    
    # Finalize session key as EB

    def complete_auth_as_EB(self, KA_B, idA, VA):
        # Compute VA_check = H2(KA_B || idA)
        idA_bytes = idA.encode()
        KA_B_bytes = KA_B.x.to_bytes(24, 'big') + KA_B.y.to_bytes(24, 'big')

        VA_check = H2(KA_B_bytes + idA_bytes, self.q)

        if VA_check != VA:
            print("[ERROR] VA verification failed at EB. Session terminated.")
            return None

        # Compute shared session key
        idB_bytes = self.idx.encode()
        SSKAB = H3(idA_bytes + idB_bytes + KA_B_bytes, self.q)
        return SSKAB


In [26]:
TA = TrustedAnchor()
params = TA.get_public_params()

print("\n TA Public Parameters:\n")
for k, v in params.items():
    print(f"{k}: {v}\n")


# Register EA
EA = Entity("meter-A", TA)
EA.register()

# Register EB
EB = Entity("meter-B", TA)
EB.register()

# Run authentication
result = EA.initiate_auth(EB)

EA_entity = result["EA_entity"]
SSKAB = result["SSKAB"]
idB = result["idB"]
RB = result["RB"]
VA = result["VA"]
KB_A = result["KB_A"]


if SSKAB:
    print(f"Session key (EA side): {SSKAB}")

    # EB completes authentication
    KA_B = KB_A  # Theorem 2
    SSKAB_EB = EB.complete_auth_as_EB(KA_B, EA.idx, VA)

    print(f"Session key (EB side): {SSKAB_EB}")

    if SSKAB == SSKAB_EB:
        print("[SUCCESS] Both sides derived the same session key!")
    else:
        print("[FAILURE] Session key mismatch.")




 TA Public Parameters:

curve_eq: y² = x³ + 6277101735386680763835789423207666416083908700390324961276x + 2455155546008943817740293915197451784769108058161191238065 mod 6277101735386680763835789423207666416083908700390324961279

p: 6277101735386680763835789423207666416083908700390324961279

q: 6277101735386680763835789423176059013767194773182842284081

P: (602046282375688656758213480587526111916698976636884684818, 174050332293622031404857552280219410364023488927386650641)

PK_TA: (4297435756938361930989519957121900037831303949182450465776, 1941290216224947029559235012819784738000544567149284489344)

H1: SHA-256

H2: SHA3-256

H3: BLAKE2s

[SUCCESS] Entity meter-A public key verified.
[SUCCESS] Entity meter-B public key verified.
Session key (EA side): 5277774916113440478062883308500972896672074892343696667903
Session key (EB side): 5277774916113440478062883308500972896672074892343696667903
[SUCCESS] Both sides derived the same session key!
